# 834. Sum of Distances in Tree

There is an undirected connected tree with n nodes labeled from 0 to n - 1 and n - 1 edges.

You are given the integer n and the array edges where edges[i] = [ai, bi] indicates that there is an edge between nodes ai and bi in the tree.

Return an array answer of length n where answer[i] is the sum of the distances between the ith node in the tree and all other nodes.

https://leetcode.com/problems/sum-of-distances-in-tree/description/

Example:
```
Input: n = 6, edges = [[0,1],[0,2],[2,3],[2,4],[2,5]]
Output: [8,12,6,10,10,10]
Explanation: 
The tree is shown above.
We can see that dist(0,1) + dist(0,2) + dist(0,3) + dist(0,4) + dist(0,5)
equals 1 + 1 + 2 + 2 + 2 = 8.
Hence, answer[0] = 8, and so on.

```


## Solution：DFS, Re-Root, DP

參考： https://github.com/wisdompeak/LeetCode/tree/master/Tree/834.Sum-of-Distances-in-Tree

這題是一個沒有cycle的連通圖
要知道任一個節點對應每個節點的距離和，可以使用「re-root」的概念<br>
主要就是要先知道根節點到每個節點的距離之和（可以深度走訪取得）<br>
然後根節點的相鄰子節點，到每個節點的距離和，就是包含自己與連接到的其他子樹節點的距離-1，與其他節點的距離+1,所以就是：<br>

非根節點對應每個節點的距離和：f(child) = f(parent) - (child子樹的節點數) + （N - child子樹的節點數）

所以還要知道每個節點的子樹節點的數量才行～（可以深度走訪取得）<br>


code執行方面：
- 為了DFS，建一個記錄走訪的的list，還有一個每個節點與相鄰節點的dictionary（{cur_node:[adjacent_nodes]}

- 為了儲存每個節點的子樹節點數量，建一個count的list，index是節點值，value是那個節點的子樹節點數量（包含自己）

- 建一個能計算子樹節點數量並填完count的dfs函數：遍歷當前節點的下一層節點，已經訪問過的就跳過，沒訪問過的，先標記在訪問清單中，然後遞迴訪問這個節點，看它有沒有下一層，已經到leaf node的話，就回傳1(子節點只算自己一個)，接著把當前節點以下的子樹節點數量通通加起來回傳，再把這個數放到count[i]之中。

- 建一個能計算根部節點到每個節點的路徑長度的總和的dfs函數：因為count有記錄到每個節點的子樹節點數量，把這個數扣1就是當前節點向下一層的路徑會被經過的次數（底下共幾個節點就會被加幾次），因此，把深度走訪過程之中的節點的「count[i]-1」全部相加起來的話，就是根節點到每一個節點的距離之總和了～～

- 依序呼叫上面建好的函數，以拿到count跟根節點到各節點的距離和，記得每次呼叫要初始化visited[]，然後走訪的第一個點要先標記為1

- 最後再建一個dfs函數，再次走訪，用動態規畫，把根節點算出的路徑總和，以「移根」的想法往子節點推，然後把每個節點的計算結果存到res[]中，這個res就是答案了

In [ ]:
from collections import defaultdict

class Solution:
    def sumOfDistancesInTree(self, n: int, edges: List[List[int]]) -> List[int]:
        # 移根的概念，先用DFS遍歷，得到root到每個節點的距離之和，以及每個節點對應的子數的節點個數
        # 非根節點對應每個節點的距離和：f(child) = f(parent) - (child子樹的節點數) + （N - child子樹的節點數）

        visited = [0 for _ in range(n)]
        count = [0 for _ in range(n)]

        # 建立每個節點與連結的節點的list
        next = defaultdict(list)
        for edge in edges:
            next[edge[0]].append(edge[1])
            next[edge[1]].append(edge[0])
        
        print(next)

        # 計算子樹節點數量(存在count中)
        def dfs(cur):
            ss = 1    # 尾端的最後子結果
            for x in next[cur]:
                if visited[x] != 0:
                    continue
                else:
                    visited[x] = 1
                    ss += dfs(x)
            count[cur] = ss
            return ss

        # 計算根節點到每個位置的距離和
        def dfs_path(cur):
            ss = 0 
            for x in next[cur]:
                if visited[x] != 0:
                    continue
                else:
                    visited[x] = 1
                    ss += dfs_path(x)
            ss += count[cur]-1   # 把每個節點往下的所有path加起來了
            return ss

        
        visited[0] = 1
        dfs(0)           #得到count

        visited = [0 for _ in range(n)]  # 訪問list歸零
        visited[0] = 1                   # 記得第一個點還是要標記已訪問！
        res = [0 for _ in range(n)]
        res[0] = dfs_path(0)   # 得到根節點到每個節點的路徑和
        print(res[0])
        print(count)

        def dfs_res(cur, total):
            for x in next[cur]:
                if visited[x] != 0:
                    continue
                else:
                    visited[x] = 1
                    a = count[x]
                    b = n - count[x]
                    res[x] = total + b - a
                    dfs_res(x, res[x])

        visited = [0 for _ in range(n)]  # 訪問list歸零
        visited[0] = 1
        dfs_res(0, res[0])

        return res

# 2503. Maximum Number of Points From Grid Queries

https://leetcode.com/problems/maximum-number-of-points-from-grid-queries/description/

You are given an m x n integer matrix grid and an array queries of size k.

Find an array answer of size k such that for each integer queres[i] you start in the top left cell of the matrix and repeat the following process:

If queries[i] is strictly greater than the value of the current cell that you are in, then you get one point if it is your first time visiting this cell, and you can move to any adjacent cell in all 4 directions: up, down, left, and right.
Otherwise, you do not get any points, and you end this process.
After the process, answer[i] is the maximum number of points you can get. Note that for each query you are allowed to visit the same cell multiple times.

Return the resulting array answer.

Example:
```
Input: grid = [[1,2,3],[2,5,7],[3,5,1]], queries = [5,6,2]
Output: [5,8,1]
Explanation: The diagrams above show which cells we visit to get points for each query.

```

## Solution：Heap queue, BFS

參考：https://github.com/wisdompeak/LeetCode/tree/master/BFS/2503.Maximum-Number-of-Points-From-Grid-Queries

主要的想法是在BFS的過程之中，優先處理值較小的相鄰位置～<br>
因此在下一層的queue使用heap queue(priority queue)存放位置，每次都優先取出最小值來看有沒有大於queries中的指定值<br>
當限制條件之內（小於指定值的數）都遍歷完並且計數過之後，pq之中存放的是下一層邊界的值。<br>
因為queries是照順序由小到大排，所以下一個指定值能覆蓋的數可以接著加，count不用歸零。

code執行：

- 先將queries的值由小到大排列，但因為最後要按照原本的index排列，所以排序的list中用tuple來存放query值與他的index，以便最後找到安放的位置

- 建一個bfs要使用的priority queue，這裡用python library: [heapq](https://python-doc-tw.github.io/library/heapq.html) ，queue之中放一個tuple，存grid中的值與座標（val, x, y)

- 建一個m*n的visited array, 記錄訪問過的位置（m = len(grid), n = len(grid[0])), 左上角第一個位置一開始就要標記=1

- 建一個方向集direct，把要走的四個方向列出來[[1,0],[-1,0],[0,1],[0,-1]]

- 建一個最後要回傳的res[], 長度和queries一樣，記錄每個指定值可以淹沒的最大格子數

- 迴圈遍歷queries，然後從左上角第一格開始進行BFS走訪，看在當前query值之下可以走訪的格子數幾個，把它存到res中

- BFS走訪過程為，當下一層的queue還有值時、以及heap_queue中的最小值還比query值小時，繼續執行，從queue最小的值開始處理（使用pop，處理完就丟掉)，處理時count++<br>
  對這個值的四個方向尋訪，走得到的就放到queue之中(heappush)，並且在visited標記已走訪。

- 因為query值有由小到大放，所以小圈的count繼續往上加就好，最後for loop完整個queries，就可以拿到result



In [ ]:
import heapq
class Solution:
    def maxPoints(self, grid: List[List[int]], queries: List[int]) -> List[int]:

        # 建一個queries_sort的list，讓這個值由小排到大
        qs = []
        for i,v in enumerate(queries):
            qs.append((v,i))
        qs.sort(key = lambda x:x[0])


        # 建一個priority_queue，初始值放grid中的當前值，與當前位置的x,y座標
        pq = [(grid[0][0], 0, 0)]
        count = 0 
        direct = [[1,0],[-1,0],[0,1],[0,-1]]
        m = len(grid)
        n = len(grid[0])
        visited = [[0 for _ in range(n)] for _ in range(m)]
        visited[0][0] = 1
        res = [0 for _ in range(len(queries))]
        
        for q,idx in qs:

            while pq and pq[0][0] < q:
                cur = heapq.heappop(pq)   # 這裡是用heap_queue(priority_queqe)!很重要～～～～
                i = cur[1]
                j = cur[2]
                count += 1

                # 找下一個位置
                for k in range(4):
                    x = i + direct[k][0]
                    y = j + direct[k][1]
                    if x < 0 or x >= m  or y < 0 or y >= n:
                        continue    # 超過邊界不繼續
                    if visited[x][y] == 1:
                        continue    # 走訪過了不繼續
                    
                    heapq.heappush(pq, (grid[x][y],x,y))
                    visited[x][y] = 1
                
            # print(pq)
            res[idx] = count
                
        return res